In [4]:
import pandas as pd

transactions = pd.read_csv('transactions.csv')

stores = pd.read_csv('stores.csv')
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
